# Langchain: The basics

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

In [1]:
!pip -q install openai langchain langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.0 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

In [3]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')


## Zero Shot Prompting



In [4]:
from langchain_openai import ChatOpenAI

In [5]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo',
             temperature=0.2,
             max_tokens = 256)

In [6]:
text = """What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:"""

print(llm.invoke(text))

content='Positive' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 52, 'total_tokens': 54, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-48db4199-8105-4bab-ae16-033d23386272-0' usage_metadata={'input_tokens': 52, 'output_tokens': 2, 'total_tokens': 54, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


## Prompting with Prompt Templates

In [ ]:
from langchain import PromptTemplate

In [ ]:
restaurant_template = """
What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: {review_text}
Sentiment:
"""

review_prompt = PromptTemplate(
    input_variables=["review_text"],
    template=restaurant_template,
)

In [ ]:
reviews = [
    "The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.",
    "Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.",
    "The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!",
    "The food tasted alright, but the tables were not very clean, which was off-putting.",
    "Waited 30 minutes even with a reservation, and the main course was served cold. Disappointing visit."
]

In [ ]:
print(review_prompt.format(review_text=reviews[0]))


What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:



In [ ]:
## querying the model with the prompt template
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=review_prompt)

<ipython-input-14-7b8f9a9dc912>:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=review_prompt)


In [ ]:
chain.invoke({"review_text": reviews[0]})

{'review_text': 'The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.',
 'text': 'Positive'}

In [ ]:
for review in reviews:
    print(review_prompt.format(review_text=review))
    print(chain.run({"review_text": review}))
    print("================================")


What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:



<ipython-input-15-fd04b0bfa63f>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run({"review_text": review}))


Positive

What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.
Sentiment:

Positive

What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!
Sentiment:

Positive

What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The food tasted alright, but the tables were not very clean, which was off-putting.
Sentiment:

Negative

What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: Waited 30 minutes even with a reservation, and the main course was served cold. Disappointing visit.
Sentiment:

Negative


## Prompting ith Few Shot Learning

In [ ]:
from langchain import FewShotPromptTemplate

In [ ]:
# First, create the list of few shot examples.
examples = [
    {
        "Review": "The grilled chicken was seasoned to perfection and simply melted in the mouth.",
        "Category": "Food Quality"
    },
    {
        "Review": "Despite the crowd, the place was immaculately clean and the restrooms were spotless.",
        "Category": "Overall Hygiene"
    },
    {
        "Review": "The dim lighting and soothing jazz music provided an intimate and romantic setting.",
        "Category": "Restaurant Ambience"
    },
    {
        "Review": "We were kept waiting for our table even after a confirmed reservation and the staff seemed disinterested.",
        "Category": "Customer Service"
    }
]

In [ ]:
# Prepare the prompt template
example_formatter_template = """
review: {Review}
category: {Category}\n
"""
category_prompt = PromptTemplate(
    input_variables=["Review", "Category"],
    template=example_formatter_template,
)

In [ ]:
print(category_prompt.format(**examples[0]))


review: The grilled chicken was seasoned to perfection and simply melted in the mouth.
category: Food Quality




In [ ]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_category_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=category_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Classify the reviews into one the four categories as given in the examples.",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Review: {review_text}\nCategory:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["review_text"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [ ]:
# We can now generate a prompt using the `format` method.
print(few_shot_category_prompt.format(review_text=reviews[0]))

Classify the reviews into one the four categories as given in the examples.

review: The grilled chicken was seasoned to perfection and simply melted in the mouth.
category: Food Quality



review: Despite the crowd, the place was immaculately clean and the restrooms were spotless.
category: Overall Hygiene



review: The dim lighting and soothing jazz music provided an intimate and romantic setting.
category: Restaurant Ambience



review: We were kept waiting for our table even after a confirmed reservation and the staff seemed disinterested.
category: Customer Service


Review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Category:


In [ ]:
from langchain.chains import LLMChain

category_chain = LLMChain(llm=llm, prompt=few_shot_category_prompt)

print(reviews[0])
# Run the chain only specifying the input variable.
print(category_chain.run(reviews[0]))

The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Food Quality, Restaurant Ambience


In [ ]:
for review in reviews:
    print(review)
    print(category_chain.run({"review_text": review}))
    print("================================")

The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Food Quality & Restaurant Ambience
Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.
Customer Service
The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!
Restaurant Ambience
The food tasted alright, but the tables were not very clean, which was off-putting.
Overall Hygiene
Waited 30 minutes even with a reservation, and the main course was served cold. Disappointing visit.
Food Quality


## Chaining Multiple Prompts

In [ ]:
from langchain.schema import StrOutputParser

In [ ]:
review_chain = review_prompt | llm | StrOutputParser()

In [ ]:
review_chain.invoke({"review_text": reviews[0]})

'Positive'

In [ ]:
category_chain = few_shot_category_prompt | llm | StrOutputParser()

In [ ]:
category_chain.invoke({"review_text": reviews[1]})

'Customer Service'

In [ ]:
response_to_customer = """
"Write an appropriate response to the customer by either appreciating their positive experience or adressing the specific concern that customer might
have mentioned in the feedback below.

<feedback>
{feedback}
</feedback>

The feedback is about {category}.

The goal of the response is to ensure customer engagement.
"""

In [ ]:
response_prompt = PromptTemplate(
    input_variables=["feedback", "category"],
    template=response_to_customer,
)

In [ ]:
final_response_chain = response_prompt | llm | StrOutputParser()

In [ ]:
complete_chain = (
    {
        "feedback": review_chain,
        "category": category_chain,
    }
    | final_response_chain
)

In [ ]:
from pprint import pprint

In [ ]:
reviews[3]

'The food tasted alright, but the tables were not very clean, which was off-putting.'

In [ ]:
pprint(complete_chain.invoke({"review_text": reviews[4]}))

('We are sorry to hear that you had a negative experience. We take customer '
 'feedback very seriously and would like to address any concerns you may have. '
 'Please feel free to reach out to us directly so we can further discuss and '
 'work towards resolving any issues you may have encountered. Thank you for '
 'bringing this to our attention.')
